In [ ]:
import requests
import time
import pathlib
import re
import pandas as pd
import numpy as np
from datetime import datetime as dt
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from sec_cik_mapper import StockMapper
import os

In [ ]:
# file paths
ROOT_OUT = pathlib.Path(
    "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/"
    "Documents/Master Finance/MasterThesis/ThesisData"
)

# russell 3000 ticker data
RUSSELL_FILE = "/Users/daniel/Library/Mobile Documents/com~apple~CloudDocs/Documents/Master Finance/MasterThesis/ThesisData/iShares-Russell-3000-ETF_fund.csv"

# api settings
UA = "VU-AM Finance Lab / daniel@vu.nl"
SLEEP = 0.12
CHECKPOINT_INTERVAL = 50

# years to process (fiscal years 2020-2024, some filing dates in 2025)
YEARS_TO_PROCESS = range(2024, 2019, -1)

In [ ]:
# load russell 3000 data
df = pd.read_csv(RUSSELL_FILE, skiprows=7)
print(f"loaded {len(df)} companies from russell 3000")

# normalize tickers
df["Ticker"] = df["Ticker"].str.upper().str.strip()

# map tickers to ciks
mapper = StockMapper()
df["cik"] = df["Ticker"].map(mapper.ticker_to_cik)

# check mapping success
mapped_count = df["cik"].notna().sum()
print(f"mapped {mapped_count} out of {len(df)} tickers to ciks")

# drop unmapped tickers
df = df.dropna(subset=['cik'])
print(f"final dataset: {len(df)} companies with valid ciks")

In [ ]:
#Define Text Cleaning Functions

def get_full_cleaned_10k_text(html_content: str) -> str:
    """clean html and xbrl markup from 10-k filing to extract readable text"""
    if not isinstance(html_content, str) or not html_content.strip():
        return ""

    processed_content_for_soup = html_content
    content_start_pattern = re.compile(r"(<\?xml|<html|<body|<!DOCTYPE html)", re.IGNORECASE)
    match = content_start_pattern.search(html_content)
    
    if match:
        text_before_match = html_content[:match.start()]
        if "<TYPE>" in text_before_match.upper() and \
           ("<TEXT>" in text_before_match.upper() or "<XBRL>" in text_before_match.upper()):
            processed_content_for_soup = html_content[match.start():]
        elif not text_before_match.strip():
            processed_content_for_soup = html_content[match.start():]
        else:
            processed_content_for_soup = html_content[match.start():]
    
    try:
        soup = BeautifulSoup(processed_content_for_soup, 'lxml')
        
        # remove unwanted tags
        for tag_to_remove in soup(["head", "title", "meta", "script", "style"]):
            tag_to_remove.decompose()
        
        # remove hidden elements
        for hidden_element in soup.find_all(style=lambda value: value and "display:none" in value.replace(" ", "").lower()):
            hidden_element.decompose()
        
        # remove xbrl wrapper tags
        common_ix_wrappers = ["ix:header", "ix:hidden", "ix:resources"]
        for ix_tag_name in common_ix_wrappers:
            for tag in soup.find_all(ix_tag_name.lower()):
                tag.decompose()
        
        xbrl_tags_to_decompose = ["xbrli:context", "xbrli:unit", "link:schemaref", 
                                  "link:linkbaseref", "link:roleref", "link:arcrolef"]
        for x_tag_name in xbrl_tags_to_decompose:
            for tag in soup.find_all(x_tag_name.lower()):
                tag.decompose()
        
        # get cleaned text
        text = soup.get_text(separator=' ', strip=True)
        text = re.sub(r'[ \t]+', ' ', text)
        text = re.sub(r'\s*\n\s*', '\n', text)
        text = text.strip()
        
    except Exception as e:
        # fallback to regex cleaning if beautifulsoup fails
        text = re.sub(r"<[^>]+>", " ", processed_content_for_soup)
        text = re.sub(r'[ \t]+', ' ', text).strip()
        text = re.sub(r'\s*\n\s*', '\n', text).strip()
    
    return text

def extract_main_10k(doc_txt: str) -> str:
    """extract the main 10-k document block from sec filing text"""
    for part in doc_txt.split("<DOCUMENT>"):
        match = re.search(r"<TYPE>\s*([\w\-/]+)", part, flags=re.I)
        if match and match.group(1).upper() == "10-K":
            return part
    
    # fallback - check whole document
    match = re.search(r"<TYPE>\s*([\w\-]+)", doc_txt, flags=re.I)
    if match and match.group(1).upper() == "10-K":
        return doc_txt
    
    return ""

In [ ]:
#Define Section Extraction Functions

def extract_risk_factors_section_v4(cleaned_full_text: str) -> str:
    """extract item 1a risk factors section from cleaned 10-k text"""
    if not isinstance(cleaned_full_text, str) or not cleaned_full_text.strip():
        return ""

    # flexible pattern for risk factors
    item_1a_heading_pattern = re.compile(
        r"Item\s*1A\s*\.?\s*" +
        r"(?:R\s*I\s*S\s*K)\s+" +
        r"(?:F\s*A\s*C\s*T\s*O\s*R\s*S)",
        re.IGNORECASE
    )
    
    # fallback pattern
    simple_rf_pattern = re.compile(
        r"^\s*(?:R\s*I\s*S\s*K)\s+(?:F\s*A\s*C\s*T\s*O\s*R\s*S)\s*$",
        re.IGNORECASE | re.MULTILINE
    )
    
    # end patterns
    end_patterns = [
        re.compile(r"Item\s*1B\s*\.?\s*Unresolved\s*Staff\s*Comments", re.IGNORECASE),
        re.compile(r"Item\s*2\s*\.?\s*Properties", re.IGNORECASE),
        re.compile(r"Item\s*3\s*\.?\s*Legal\s*Proceedings", re.IGNORECASE),
        re.compile(r"PART\s*II", re.IGNORECASE)
    ]

    candidate_sections = []
    search_patterns_for_start = [item_1a_heading_pattern, simple_rf_pattern]

    for start_pattern_idx, start_pattern in enumerate(search_patterns_for_start):
        for start_match in start_pattern.finditer(cleaned_full_text):
            # skip table of contents entries
            potential_toc_indicators_text = cleaned_full_text[start_match.end() : start_match.end() + 100]
            if re.match(r"^\s*\.{3,}", potential_toc_indicators_text):
                continue
            
            line_start_index = cleaned_full_text.rfind('\n', 0, start_match.start()) + 1
            line_end_index = cleaned_full_text.find('\n', start_match.end())
            if line_end_index == -1:
                line_end_index = len(cleaned_full_text)
            
            current_line_of_match = cleaned_full_text[line_start_index:line_end_index]
            text_on_line_after_heading = current_line_of_match[start_match.end() - line_start_index:]
            
            # check for page numbers
            cond1_dots_and_page = re.search(r"^\s*\.{3,}\s*\d+\s*$", text_on_line_after_heading)
            cond2_just_page = re.match(r"^\s*[ivxlcdm\d]+(\b|\s|$)", text_on_line_after_heading.lower())
            if cond1_dots_and_page or cond2_just_page:
                continue

            content_start_index = start_match.end()
            text_after_heading = cleaned_full_text[content_start_index:]
            min_end_idx_in_slice = len(text_after_heading)
            end_pattern_found = False
            
            for end_pattern in end_patterns:
                end_match = end_pattern.search(text_after_heading)
                if end_match:
                    if end_match.start() < min_end_idx_in_slice:
                        min_end_idx_in_slice = end_match.start()
                    end_pattern_found = True
            
            if not end_pattern_found:
                continue

            current_extracted_section = text_after_heading[:min_end_idx_in_slice].strip()
            if len(current_extracted_section) > 50:
                candidate_sections.append(current_extracted_section)
        
        if candidate_sections and start_pattern_idx == 0:
            break
    
    if not candidate_sections:
        return ""
    
    return max(candidate_sections, key=len)

def extract_item7_mda(cleaned_full_text: str) -> str:
    """extract item 7 management discussion and analysis from cleaned 10-k text"""
    if not isinstance(cleaned_full_text, str) or not cleaned_full_text.strip():
        return ""

    item7_heading_pattern = re.compile(
        r"Item\s*7\s*\.\s*Management['']?s\s*Discussion\s*and\s*Analysis\s*of\s*Financial\s*Condition\s*and\s*Results\s*of\s*Operations",
        re.IGNORECASE
    )
    
    simple_mda_heading_pattern = re.compile(
        r"^\s*MANAGEMENT['']?S\s*DISCUSSION\s*AND\s*ANALYSIS\s*(?:OF\s*FINANCIAL\s*CONDITION\s*AND\s*RESULTS\s*OF\s*OPERATIONS)?\s*$",
        re.IGNORECASE | re.MULTILINE
    )
    
    end_patterns = [
        re.compile(r"Item\s*7A\s*\.?\s*Quantitative\s*and\s*Qualitative\s*Disclosures\s*About\s*Market\s*Risk", re.IGNORECASE),
        re.compile(r"Item\s*8\s*\.?\s*Financial\s*Statements\s*and\s*Supplementary\s*Data", re.IGNORECASE),
        re.compile(r"PART\s*III", re.IGNORECASE),
        re.compile(r"SIGNATURES", re.IGNORECASE)
    ]

    candidate_sections = []
    search_patterns_for_start = [item7_heading_pattern, simple_mda_heading_pattern]

    for start_pattern_idx, start_pattern in enumerate(search_patterns_for_start):
        for start_match in start_pattern.finditer(cleaned_full_text):
            # skip table of contents
            potential_toc_indicators_text = cleaned_full_text[start_match.end() : start_match.end() + 100]
            if re.match(r"^\s*\.{3,}", potential_toc_indicators_text):
                continue
            
            line_start_index = cleaned_full_text.rfind('\n', 0, start_match.start()) + 1
            line_end_index = cleaned_full_text.find('\n', start_match.end())
            if line_end_index == -1:
                line_end_index = len(cleaned_full_text)
            
            current_line_of_match = cleaned_full_text[line_start_index:line_end_index]
            text_on_line_after_heading = current_line_of_match[start_match.end() - line_start_index:]
            
            cond1_dots_and_page = re.search(r"^\s*\.{3,}\s*\d+\s*$", text_on_line_after_heading)
            cond2_just_page = re.match(r"^\s*[ivxlcdm\d]+(\b|\s|$)", text_on_line_after_heading.lower())
            if cond1_dots_and_page or cond2_just_page:
                continue

            content_start_index = start_match.end()
            text_after_heading = cleaned_full_text[content_start_index:]
            min_end_idx_in_slice = len(text_after_heading)
            end_pattern_found = False
            
            for end_pattern in end_patterns:
                end_match = end_pattern.search(text_after_heading)
                if end_match:
                    if end_match.start() < min_end_idx_in_slice:
                        min_end_idx_in_slice = end_match.start()
                    end_pattern_found = True
            
            if not end_pattern_found:
                continue

            current_extracted_section = text_after_heading[:min_end_idx_in_slice].strip()
            if len(current_extracted_section) > 50:
                candidate_sections.append(current_extracted_section)
        
        if candidate_sections and start_pattern_idx == 0:
            break

    if not candidate_sections:
        return ""
    
    return max(candidate_sections, key=len)

In [ ]:
# setup requests session
session = requests.Session()
session.headers.update({"User-Agent": UA, "Accept-Encoding": "gzip, deflate"})

def fetch_json(url: str) -> dict | None:
    """fetch json data from sec api"""
    try:
        r = session.get(url, headers={"Host": "data.sec.gov"})
        r.raise_for_status()
        return r.json()
    except requests.exceptions.RequestException as e:
        tqdm.write(f"json error {url}: {e}")
        return None

def fetch_txt(url: str) -> str | None:
    """fetch text data from sec website"""
    try:
        r = session.get(url)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        tqdm.write(f"text error {url}: {e}")
        return None

In [ ]:
# column mappings
USER_CIK_COLUMN = 'cik'
USER_TICKER_COLUMN = 'Ticker'
USER_COMPANY_NAME_COLUMN = 'Name'
USER_SECTOR_COLUMN = 'Sector'

# prepare cik dataframe
df_ciks_input = df[[USER_CIK_COLUMN, USER_TICKER_COLUMN, USER_COMPANY_NAME_COLUMN, USER_SECTOR_COLUMN]].copy()
df_ciks_input[USER_CIK_COLUMN] = df_ciks_input[USER_CIK_COLUMN].astype(str).str.zfill(10)
df_ciks_input.drop_duplicates(subset=[USER_CIK_COLUMN], keep='first', inplace=True)

print(f"processing {len(df_ciks_input)} unique ciks")

In [ ]:
#Retrieve 10-K Filings for All Years

# main loop to retrieve 10-k filings
ROOT_OUT.mkdir(parents=True, exist_ok=True)

for current_year in tqdm(YEARS_TO_PROCESS, desc="processing years"):
    tqdm.write(f"\nprocessing year: {current_year}")
    output_folder_year = ROOT_OUT / str(current_year)
    output_folder_year.mkdir(parents=True, exist_ok=True)
    
    # file paths for this year
    checkpoint_path = output_folder_year / f"10k_fullcleaned_info_{current_year}_checkpoint.parquet"
    final_path = output_folder_year / f"10k_fullcleaned_info_{current_year}_final.parquet"

    records_for_year = []
    processed_filing_ids_this_year = set()

    # load checkpoint if exists
    if checkpoint_path.exists():
        try:
            df_checkpoint = pd.read_parquet(checkpoint_path)
            records_for_year = df_checkpoint.to_dict('records')
            for record in records_for_year:
                if USER_CIK_COLUMN in record and 'accession' in record:
                    processed_filing_ids_this_year.add((str(record[USER_CIK_COLUMN]).zfill(10), record['accession']))
            tqdm.write(f"resumed for year {current_year}. loaded {len(records_for_year)} records")
        except Exception as e:
            tqdm.write(f"error loading checkpoint: {e}. starting fresh for {current_year}")
            records_for_year, processed_filing_ids_this_year = [], set()

    newly_added_count_this_session = 0
    
    for index, row in tqdm(df_ciks_input.iterrows(), total=len(df_ciks_input), desc=f"ciks for {current_year}", leave=False):
        cik_from_df = str(row[USER_CIK_COLUMN]).zfill(10)
        ticker_from_df = row[USER_TICKER_COLUMN]
        company_name_from_df = row[USER_COMPANY_NAME_COLUMN]
        sector_from_df = row[USER_SECTOR_COLUMN]
        
        # fetch submissions data
        sub_url = f"https://data.sec.gov/submissions/CIK{cik_from_df}.json"
        submission_data = fetch_json(sub_url)
        time.sleep(SLEEP)
        
        if not submission_data:
            continue
        
        # get company info from sec
        company_name_sec = submission_data.get('name', 'N/A')
        tickers_list_sec = submission_data.get('tickers', [])
        tickers_sec_str = ", ".join(tickers_list_sec) if tickers_list_sec else 'N/A'

        if "filings" not in submission_data or "recent" not in submission_data["filings"]:
            continue
            
        filings = submission_data["filings"]["recent"]
        req_keys = ["form", "accessionNumber", "filingDate"]
        
        if not all(k in filings for k in req_keys) or \
           not all(isinstance(filings[k], list) for k in req_keys) or \
           (len(filings["form"]) > 0 and len(set(len(filings[k]) for k in req_keys)) > 1):
            continue

        # process each filing
        for i in range(len(filings["form"])):
            form_type = filings["form"][i]
            acc_num = filings["accessionNumber"][i]
            fdate_str = filings["filingDate"][i]
            
            if form_type not in {"10-K", "10-K/A"}:
                continue
                
            try:
                filed_date_obj = dt.strptime(fdate_str, "%Y-%m-%d").date()
            except ValueError:
                continue
                
            if filed_date_obj.year != current_year or (cik_from_df, acc_num) in processed_filing_ids_this_year:
                continue
            
            # download filing
            acc_nodash = acc_num.replace("-", "")
            filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik_from_df)}/{acc_nodash}/{acc_num}.txt"
            raw_filing_text = fetch_txt(filing_url)
            time.sleep(SLEEP)
            
            if not raw_filing_text:
                continue

            # extract main 10-k document
            extracted_block = extract_main_10k(raw_filing_text)
            if not extracted_block:
                continue
            
            # get cleaned text
            full_cleaned_text = get_full_cleaned_10k_text(extracted_block)
            
            if not full_cleaned_text.strip():
                tqdm.write(f"cik {cik_from_df}, acc {acc_num}, year {current_year}: empty text after cleaning")
                continue
            
            # save record
            records_for_year.append({
                "cik": cik_from_df,
                "companyName_user": company_name_from_df,
                "ticker_user": ticker_from_df,
                "sector_user": sector_from_df,
                "companyName_sec": company_name_sec,
                "tickers_sec": tickers_sec_str,
                "filingDate": fdate_str,
                "accession": acc_num,
                "form": form_type,
                "year": current_year,
                "text": full_cleaned_text,
            })
            
            processed_filing_ids_this_year.add((cik_from_df, acc_num))
            newly_added_count_this_session += 1

            # save checkpoint periodically
            if newly_added_count_this_session > 0 and newly_added_count_this_session % CHECKPOINT_INTERVAL == 0 and records_for_year:
                try:
                    df_chkp = pd.DataFrame(records_for_year).drop_duplicates(subset=['cik', 'accession'], keep='last')
                    df_chkp.to_parquet(checkpoint_path, index=False, compression='zstd')
                    tqdm.write(f"checkpoint for {current_year}: {len(df_chkp)} records saved")
                except Exception as e:
                    try:
                        df_chkp.to_parquet(checkpoint_path, index=False, compression='gzip')
                        tqdm.write(f"checkpoint (gzip) for {current_year}: {len(df_chkp)} records saved")
                    except Exception as e2:
                        tqdm.write(f"error saving checkpoint: {e2}")
    
    # save final data for year
    if records_for_year:
        try:
            df_final = pd.DataFrame(records_for_year).drop_duplicates(subset=['cik', 'accession'], keep='last')
            df_final.to_parquet(final_path, index=False, compression='zstd')
            tqdm.write(f"\nyear {current_year} complete: {len(df_final)} filings saved to {final_path}")
        except Exception as e:
            try:
                df_final.to_parquet(final_path, index=False, compression='gzip')
                tqdm.write(f"\nyear {current_year} complete (gzip): {len(df_final)} filings saved")
            except Exception as e2:
                tqdm.write(f"error saving final data for year {current_year}: {e2}")
    else:
        tqdm.write(f"\nyear {current_year}: no new filings processed")

tqdm.write("\nall years processed")

In [ ]:
#Process Year Data and Extract Sections

def process_year_data(year_int: int, base_file_path_template: str) -> pd.DataFrame | None:
    """load and process data for a single year, extract risk factors and mda sections"""
    year_str = str(year_int)
    file_path = base_file_path_template.format(year=year_str)

    columns_to_return_schema = [
        'cik', 'companyName_user', 'ticker_user', 'sector_user', 
        'companyName_sec', 'tickers_sec', 'filingDate', 'accession', 
        'form', 'year', 'risk_factors_text', 'item7_mda_text'
    ]
    
    if not os.path.exists(file_path):
        print(f"file not found for year {year_str}")
        return None

    print(f"processing data for {year_str}...")
    try:
        df = pd.read_parquet(file_path)
        
        if 'cik' not in df.columns:
            print(f"error: 'cik' column missing for {year_str}")
            return pd.DataFrame(columns=columns_to_return_schema)
            
        if 'text' not in df.columns:
            print(f"error: 'text' column missing for {year_str}")
            return pd.DataFrame(columns=columns_to_return_schema)

        print(f"loaded {file_path}")
        df['year'] = year_int

        # extract sections
        print(f"extracting risk factors for {year_str}...")
        df['risk_factors_text'] = df['text'].apply(extract_risk_factors_section_v4)
        
        print(f"extracting md&a for {year_str}...")
        df['item7_mda_text'] = df['text'].apply(extract_item7_mda)
        
        # filter out rows where extraction failed
        is_risk_missing = df['risk_factors_text'].isna() | df['risk_factors_text'].str.strip().eq("")
        is_mda_missing = df['item7_mda_text'].isna() | df['item7_mda_text'].str.strip().eq("")
        df_cleaned = df[~(is_risk_missing | is_mda_missing)].copy()
        
        if df_cleaned.empty:
            print(f"no valid rows for {year_str} after extraction")
            return pd.DataFrame(columns=columns_to_return_schema)

        # select final columns
        final_cols = [col for col in columns_to_return_schema if col in df_cleaned.columns]
        df_for_return = df_cleaned[list(dict.fromkeys(final_cols))]

        print(f"finished {year_str}: {len(df_for_return)} rows after cleaning")
        return df_for_return

    except Exception as e:
        print(f"error processing {year_str}: {e}")
        return pd.DataFrame(columns=columns_to_return_schema)

In [ ]:
#build dataset

# build panel data from all years
target_years = [2024, 2023, 2022, 2021, 2020]
base_path_template = str(ROOT_OUT / "{year}/10k_fullcleaned_info_{year}_final.parquet")

processed_yearly_dfs = {}
all_target_year_files_found = True

for year_val in target_years:
    print(f"\nprocessing year: {year_val}")
    df_year_processed = process_year_data(year_val, base_path_template)
    
    if df_year_processed is None:
        all_target_year_files_found = False
        print(f"critical: file for year {year_val} not found")
        break
        
    processed_yearly_dfs[year_val] = df_year_processed

panel_df = pd.DataFrame()

if not all_target_year_files_found:
    print("\nnot all year files were found")
elif len(processed_yearly_dfs) != len(target_years):
    print(f"\nprocessing error: expected {len(target_years)} dataframes, got {len(processed_yearly_dfs)}")
else:
    print(f"\nall {len(target_years)} year files processed. identifying common ciks...")
    
    # find ciks present in all years
    list_of_cik_sets = []
    for year_val in target_years:
        df = processed_yearly_dfs[year_val]
        if 'cik' in df.columns and not df['cik'].dropna().empty:
            list_of_cik_sets.append(set(df['cik'].dropna().unique()))
        else:
            list_of_cik_sets.append(set())
            print(f"warning: year {year_val} has no valid ciks")

    if len(list_of_cik_sets) == len(target_years):
        common_ciks = list_of_cik_sets[0]
        for cik_set in list_of_cik_sets[1:]:
            common_ciks.intersection_update(cik_set)

        print(f"found {len(common_ciks)} ciks present in all {len(target_years)} years")

        if common_ciks:
            # combine data for common ciks
            final_data_to_concat = []
            for year_val in target_years:
                df_year = processed_yearly_dfs[year_val]
                if 'cik' in df_year.columns:
                    filtered_df = df_year[df_year['cik'].isin(common_ciks)].copy()
                    final_data_to_concat.append(filtered_df)
            
            if final_data_to_concat and len(final_data_to_concat) == len(target_years):
                panel_df = pd.concat(final_data_to_concat, ignore_index=True)
                print(f"concatenated data: {len(panel_df)} total rows")

                if not panel_df.empty:
                    # sort by cik and year
                    panel_df['year'] = pd.to_numeric(panel_df['year'])
                    panel_df.sort_values(by=['cik', 'year'], ascending=[True, False], inplace=True)
                    print("panel data sorted")

                    # verify all ciks have correct number of years
                    cik_counts = panel_df.groupby('cik').size()
                    ciks_with_unexpected_counts = cik_counts[cik_counts != len(target_years)]
                    if not ciks_with_unexpected_counts.empty:
                        print(f"\nwarning: some ciks do not have exactly {len(target_years)} entries")
                        print(ciks_with_unexpected_counts)
                    else:
                        print(f"\nverified: all {panel_df['cik'].nunique()} ciks have {len(target_years)} years of data")

In [ ]:
# save the initial panel data
output_filename = "panel_data_2020_2024_strict_5years.parquet"
output_full_path = ROOT_OUT / output_filename

if not panel_df.empty:
    try:
        panel_df.to_parquet(output_full_path, index=False)
        print(f"\npanel data saved to {output_full_path}")
        print(f"final panel: {len(panel_df)} rows, {panel_df['cik'].nunique()} unique ciks")
    except Exception as e:
        print(f"error saving panel data: {e}")
else:
    print("\npanel data is empty, nothing saved")

In [ ]:
# check for duplicate filings (some ciks may have multiple 10-k filings per year)
print("\nchecking for duplicate filings...")

# reload the panel data
panel_df = pd.read_parquet(output_full_path)
print(f"loaded panel data: {len(panel_df)} rows")

# check for ciks with more than expected entries
cik_counts = panel_df.groupby('cik').size()
problematic_ciks = cik_counts[cik_counts > 5]

if not problematic_ciks.empty:
    print(f"found {len(problematic_ciks)} ciks with more than 5 entries")
    print(problematic_ciks.head())
else:
    print("no duplicate issues found")

In [ ]:
# deduplicate by keeping most recent filing per cik-year
panel_df['filingDate'] = pd.to_datetime(panel_df['filingDate'])
print("converted filing dates to datetime")

# sort by cik, year, and filing date (most recent first)
panel_df_sorted = panel_df.sort_values(
    by=['cik', 'year', 'filingDate'],
    ascending=[True, True, False]
)

# keep only the most recent filing for each cik-year combination
panel_df_deduplicated = panel_df_sorted.drop_duplicates(
    subset=['cik', 'year'],
    keep='first'
).reset_index(drop=True)

print(f"deduplication complete: {len(panel_df)} -> {len(panel_df_deduplicated)} rows")

# verify all ciks now have exactly 5 entries
cik_counts_after = panel_df_deduplicated.groupby('cik').size()
problematic_after = cik_counts_after[cik_counts_after != 5]

if problematic_after.empty:
    print(f"success: all {panel_df_deduplicated['cik'].nunique()} ciks have exactly 5 entries")
else:
    print(f"warning: {len(problematic_after)} ciks still have incorrect entry counts")

In [ ]:
# calculate text lengths for filtering
print("\ncalculating text lengths...")

if 'risk_factors_text' in panel_df_deduplicated.columns:
    panel_df_deduplicated['len_risk_factors'] = panel_df_deduplicated['risk_factors_text'].str.len()
    print("calculated risk factors text length")

if 'item7_mda_text' in panel_df_deduplicated.columns:
    panel_df_deduplicated['len_item7_mda'] = panel_df_deduplicated['item7_mda_text'].str.len()
    print("calculated md&a text length")

# combined length
if 'len_risk_factors' in panel_df_deduplicated.columns and 'len_item7_mda' in panel_df_deduplicated.columns:
    panel_df_deduplicated['combined_len_rf_mda'] = (
        panel_df_deduplicated['len_risk_factors'] + panel_df_deduplicated['len_item7_mda']
    )
    print("calculated combined text length")

# show length statistics
print("\ntext length statistics:")
print(panel_df_deduplicated[['len_risk_factors', 'len_item7_mda', 'combined_len_rf_mda']].describe())

# filter out ciks with extremely long combined text (>200k characters)
threshold = 200000

# find ciks exceeding threshold
max_combined_len_per_cik = panel_df_deduplicated.groupby('cik')['combined_len_rf_mda'].max()
ciks_exceeding_threshold = max_combined_len_per_cik[max_combined_len_per_cik > threshold]
ciks_to_exclude = ciks_exceeding_threshold.index.tolist()

print(f"found {len(ciks_to_exclude)} ciks with combined text > {threshold:,} characters")

# create filtered dataset
panel_df_filtered = panel_df_deduplicated[
    ~panel_df_deduplicated['cik'].isin(ciks_to_exclude)
].copy()

print(f"\nfiltering results:")
print(f"original ciks: {panel_df_deduplicated['cik'].nunique()}")
print(f"remaining ciks: {panel_df_filtered['cik'].nunique()}")
print(f"original rows: {len(panel_df_deduplicated)}")
print(f"remaining rows: {len(panel_df_filtered)}")

# show updated statistics
print("\nfiltered text length statistics:")
print(panel_df_filtered[['len_risk_factors', 'len_item7_mda', 'combined_len_rf_mda']].describe())

In [ ]:
# save deduplicated panel data
output_deduplicated_path = ROOT_OUT / "panel_data_2020_2024.parquet"
panel_df_deduplicated.to_parquet(output_deduplicated_path, index=False)
print(f"saved deduplicated panel data to: {output_deduplicated_path}")

# save filtered panel data (with text length constraint)
panel_data_charactermax200k = panel_df_filtered.copy()
output_filtered_path = ROOT_OUT / "panel_data_charactermax200k.parquet"

try:
    panel_data_charactermax200k.to_parquet(output_filtered_path, index=False)
    print(f"\nsaved filtered panel data to: {output_filtered_path}")
    print(f"rows saved: {len(panel_data_charactermax200k)}")
    print(f"unique ciks: {panel_data_charactermax200k['cik'].nunique()}")
    
    # final verification
    cik_counts_final = panel_data_charactermax200k.groupby('cik').size()
    if not cik_counts_final[cik_counts_final != 5].empty:
        print("\nwarning: some ciks in final dataset don't have exactly 5 entries")
    else:
        print(f"\nverified: all {panel_data_charactermax200k['cik'].nunique()} ciks have exactly 5 years of data")
        
except Exception as e:
    print(f"error saving filtered data: {e}")

# load and verify the final saved data
df = pd.read_parquet(output_filtered_path)

print(f"successfully loaded final dataset from: {output_filtered_path}")
print(f"shape: {df.shape}")
print(f"unique ciks: {df['cik'].nunique()}")
print(f"\ncolumns: {list(df.columns)}")

# show sample data
print("\nsample data (first 5 rows):")
print(df[['cik', 'ticker_user', 'companyName_user', 'year', 'filingDate', 
         'len_risk_factors', 'len_item7_mda', 'combined_len_rf_mda']].head())

# year distribution
print("\nyear distribution:")
print(df['year'].value_counts().sort_index())

# sector distribution
print("\nsector distribution:")
print(df['sector_user'].value_counts().head(10))